In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams

In [ ]:
eng_stopwords = set(stopwords.words('english'))

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
print(train_df.shape)
print(test_df.shape)

In [ ]:
def get_unigrams(que):
    return [word for word in word_tokenize(que.lower()) if word not in eng_stopwords]

def get_common_unigrams(row):
    return len( set(row["unigrams_ques1"]).intersection(set(row["unigrams_ques2"])) )

def get_common_unigram_ratio(row):
    return float(row["unigrams_common_count"]) / max(len( set(row["unigrams_ques1"]).union(set(row["unigrams_ques2"])) ),1)

train_df["unigrams_ques1"] = train_df['question1'].apply(lambda x: get_unigrams(str(x)))
train_df["unigrams_ques2"] = train_df['question2'].apply(lambda x: get_unigrams(str(x)))
train_df["unigrams_common_count"] = train_df.apply(lambda row: get_common_unigrams(row),axis=1)
train_df["unigrams_common_ratio"] = train_df.apply(lambda row: get_common_unigram_ratio(row), axis=1)

In [ ]:
test_df["unigrams_ques1"] = test_df['question1'].apply(lambda x: get_unigrams(str(x)))
test_df["unigrams_ques2"] = test_df['question2'].apply(lambda x: get_unigrams(str(x)))
test_df["unigrams_common_count"] = test_df.apply(lambda row: get_common_unigrams(row),axis=1)
test_df["unigrams_common_ratio"] = test_df.apply(lambda row: get_common_unigram_ratio(row), axis=1)

In [ ]:
submission = pd.DataFrame(test_df['test_id'])
submission['is_duplicate'] = test_df["unigrams_common_ratio"]
submission.head()

In [ ]:
submission.to_csv('common_unigrams_ratio_model.csv', index=False)